In [1]:
from src.gather_votes import gather_vote_urls

gather_vote_urls()

2025-05-01 13:06:35.874 | INFO     | src.gather_votes:gather_vote_urls:129 - Scraping vote URLs from Bundestag website...
Scraping: 930vote [00:26, 35.57vote/s]


In [1]:
import pandas as pd

df = pd.read_parquet("data/parquet/votes.parquet")

In [1]:
from src.gather_vote_texts import gather_vote_texts

gather_vote_texts()

2025-05-01 13:22:44.622 | INFO     | src.gather_vote_texts:gather_vote_texts:78 - Gathering vote texts from Bundestag website...
2025-05-01 13:22:44.660 | INFO     | src.gather_vote_texts:download_vote_text:59 - Downloading https://dserver.bundestag.de/btd/20/151/2015117.pdf to data/tmp/texts/66e2e55e-85a8-4b05-9182-d37e232e5e43/66e2e55e-85a8-4b05-9182-d37e232e5e43_0.pdf
2025-05-01 13:22:44.661 | INFO     | src.gather_vote_texts:download_vote_text:65 - File already exists: data/tmp/texts/66e2e55e-85a8-4b05-9182-d37e232e5e43/66e2e55e-85a8-4b05-9182-d37e232e5e43_0.pdf
2025-05-01 13:22:44.661 | INFO     | src.gather_vote_texts:download_vote_text:59 - Downloading https://dserver.bundestag.de/btd/20/150/2015096.pdf to data/tmp/texts/66e2e55e-85a8-4b05-9182-d37e232e5e43/66e2e55e-85a8-4b05-9182-d37e232e5e43_1.pdf
2025-05-01 13:22:44.662 | INFO     | src.gather_vote_texts:download_vote_text:65 - File already exists: data/tmp/texts/66e2e55e-85a8-4b05-9182-d37e232e5e43/66e2e55e-85a8-4b05-9182-d3

In [6]:
df.iloc[0]["pdf_url"]

'https://www.bundestag.de/resource/blob/1057522/3108b8ae92d6ef722d4a8ca803af0321/20250318_3.pdf'